NOTA:
-----

Ejecute la celda a continuación antes de cualquier otra cosa.

In [14]:
%load_ext sql
%sql sqlite:///dataset_1.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: None@dataset_1.db'

Parte 1
-------

Funciones de agregación y `GROUP BY`

Ejercicio #1
-------------

Considere un conjunto de tablas utilizadas por la industria de fabricación de bagels. Para comenzar estudiemos las siguientes dos tablas: `bagel`, que describe los tipos de bagels hechos por las diferentes compañías:
> * name STRING
> * price FLOAT
> * made_by STRING

Y `purchase`:
> * bagel_name STRING
> * franchise STRING
> * date INT
> * quantity INT
> * purchaser_age INT

En donde `purchase.bagel_name` hace referencia a `bagel.name` y `purchase.franchise` hace referencia a `bagel.made_by`:

In [32]:
%sql SELECT * FROM bagel;

Done.


name,price,made_by
Plain with shmear,1.99,Bobs Bagels
Egg with shmear,2.39,Bobs Bagels
eBagel Drinkable Bagel,27.99,eBagel
eBagel Expansion Pack,1.99,eBagel
Plain with shmear,0.99,BAGEL CORP
Organic Flax-seed bagel chips,0.99,BAGEL CORP


In [33]:
%sql SELECT * FROM purchase;

Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
eBagel Expansion Pack,eBagel,1,137,5
Plain with shmear,Bobs Bagels,4,24,None
Moonshine,BAGEL CORP,7,1000,37


     Escriba a continuación un *query* para obtener el total de ingresos para cada tipo de bagel **que tenga un promedio de edad de compradores de arriba de 18 años**:

In [20]:
%%sql

SELECT b.name, AVG(p.purchaser_age), SUM(p.quantity * b.price)
FROM purchase p, bagel b
WHERE p.bagel_name = b.name
GROUP BY b.name
HAVING AVG(p.purchaser_age) > 18;

Done.


name,AVG(p.purchaser_age),SUM(p.quantity * b.price)
Egg with shmear,47.0,14.34
Plain with shmear,23.0,146.01999999999998


In [4]:
"""
Este debería ser el resultado:

(No ejecute nuevamente esta celda)
"""

Done.


name,Revenue
Egg with shmear,14.34
Plain with shmear,84.51


In [28]:
%%sql

SELECT SUM(p.quantity * b.price)
FROM purchase p
    LEFT JOIN bagel b ON p.bagel_name = b.name
WHERE p.bagel_name = 'Plain with shmear'
    AND p.purchaser_age IS NOT NULL;

Done.


SUM(p.quantity * b.price)
74.49999999999999


In [30]:
%%sql

SELECT AVG(p.purchaser_age) FROM purchase p WHERE p.bagel_name = 'Moonshine';

Done.


AVG(p.purchaser_age)
37.0


In [31]:
%%sql
SELECT * FROM purchase;

Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
eBagel Expansion Pack,eBagel,1,137,5
Plain with shmear,Bobs Bagels,4,24,None
Moonshine,BAGEL CORP,7,1000,37


Ejercicio #2
-------------

A continuación utilizaremos una versión simplificada de la tabla `precipitation_full`, que contiene solo la cantidad diaria de lluvia medida en el estado de California, con el siguiente esquema:

> * station_id
> * day
> * precipitation

In [5]:
%sql SELECT * FROM precipitation LIMIT 5;

Done.


station_id,day,precipitation
16102,1,10
16102,4,10
16102,24,30
21201,1,0
21201,20,10


Escriba a continuación un _query_ que obtenga los ids de las estaciones que tengan un promedio de precipitación mayor a 75 **utilizando subqueries**:

In [6]:
"""
Este debería ser el resultado:

(No ejecute nuevamente esta celda)
"""

Done.


station_id
88302
250002
335701
357302
488301


Ahora re escribalo utilizando `GROUP BY`:

In [7]:
"""
Este debería ser el resultado:

(No ejecute nuevamente esta celda)
"""

Done.


station_id
88302
250002
335701
357302
488301


A continuación vamos a cronometrarlo utilizando `%%time` seguido por los queries trabajados:

**Nota:** Sí, ahí van las respuestas. Aún así es valioso que las trabaje por su cuenta.

In [9]:
%%time
%%sql

SELECT DISTINCT p.station_id
FROM precipitation p
WHERE (
    SELECT AVG(precipitation)
    FROM precipitation
    WHERE station_id = p.station_id
) > 75;

Done.
Wall time: 33.5 ms


station_id
88302
250002
335701
357302
488301


In [11]:
%%time
%%sql

SELECT p.station_id
FROM precipitation p
GROUP BY p.station_id
HAVING AVG(p.precipitation) > 75;

Done.
Wall time: 4 ms


station_id
88302
250002
335701
357302
488301


¿Cuántas veces más rápido es el tiempo de ejecución utilizando `GROUP BY`?

Parte 2
-------

Cuantificadores, NULLs y outer joins

Ejercicio #1
------------

De las tablas `bagel` y `purchase` descritas anteriormente, ¿puede determinar si existen compras de productos que no se encuentran en el listado oficial (es decir, que no se encuentran en la tabla `bagel`), sin utilizar _subqueries_?

Escriba el query a continuación: